# Text generation using RNN - Character Level

To generate text using RNN, we need a to convert raw text to a supervised learning problem format.

Take, for example, the following corpus:

"Her brother shook his head incredulously"

First we need to divide the data into tabular format containing input (X) and output (y) sequences. In case of a character level model, the X and y will look like this:

|      X     |  Y  |
|------------|-----|
|    Her b   |  r  |
|    er br   |  o  |
|    r bro   |  t  |
|     brot   |  h  |
|    broth   |  e  |
|    .....   |  .  |
|    .....   |  .  |
|    ulous   |  l  |
|    lousl   |  y  |

Note that in the above problem, the sequence length of X is five characters and that of y is one character. Hence, this is a many-to-one architecture. We can, however, change the number of input characters to any number of characters depending on the type of problem.

A model is trained on such data. To generate text, we simply give the model any five characters using which it predicts the next character. Then it appends the predicted character to the input sequence (on the extreme right of the sequence) and discards the first character (character on extreme left of the sequence). Then it predicts again using the new sequence and the cycle continues until a fix number of iterations. An example is shown below:

Seed text: "incre"

|      X                                            |  Y                       |
|---------------------------------------------------|--------------------------|
|                        incre                      |    < predicted char 1 >  |
|               ncre < predicted char 1 >              |    < predicted char 2 >  |
|       cre< predicted char 1 > < predicted char 2 >   |    < predicted char 3 >  |
|       re< predicted char 1 >< predicted char 2 > < predicted char 3 >   |    < predicted char 4 >  |
|                      ...                          |            ...           |

# Notebook Overview
1. Preprocess data
2. LSTM model
3. Generate code

In [1]:
# import libraries
import warnings
warnings.filterwarnings("ignore")

import os
import re
import numpy as np
import random
import sys
import io
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Activation, LSTM
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import get_file

# 1. Preprocess data

We're going to build a C code generator by training an RNN on a huge corpus of C code (the linux kernel code). You can download the C code used as source text from the following link:
https://github.com/torvalds/linux/tree/master/kernel

We have already downloaded the entire kernel folder and stored in a local directory

## Load C code

In [2]:
# set path where C files reside

path = '/Users/nanditapuri/Desktop/Notes/Deep Learning/RNN c-code generation'

os.chdir(path)

file_names = os.listdir()
print(file_names)

['bpf_preload_umd_blob.S', 'ftrace.c', 'tsacct.c', 'lock_events.c', 'lock_events_list.h', 'latencytop.c', 'trace_events_hist.c', 'ntp.c', 'posix-timers.h', 'rcuscale.c', 'trace_sched_switch.c', 'callchain.c', 'swiotlb.c', 'audit.h', 'prog_iter.c', 'common.c', 'arraymap.c', 'msi.c', 'trace.c', 'gcc_4_7.c', 'tick-sched.h', 'timekeeping_debug.c', 'iomem.c', 'kvm.c', 'braille.c', 'kexec_core.c', 'Kconfig.debug', 'test_udelay.c', 'bpf_inode_storage.c', 'power.h', 'energy_model.c', 'trace_kprobe_selftest.c', 'spinlock_debug.c', 'rtmutex_common.h', 'async.c', 'power-traces.c', 'utsname.c', 'queue_stack_maps.c', 'dispatcher.c', 'trace_synth.h', 'rseq.c', 'rcu_segcblist.h', 'kcsan.h', 'internal.h', 'debug.h', 'core.h', 'timer.c', 'tree_plugin.h', 'trace_irqsoff.c', 'trace_events.c', 'trace_nop.c', 'net_namespace.c', 'module_signing.c', 'shadow.c', 'process.c', 'cputime.c', 'trace_selftest.c', 'trace_functions.c', 'exec_domain.c', 'smpboot.c', '.DS_Store', 'cpupri.h', 'patch.c', 'core_sched.c', 

In [3]:
# use regex to filter .c files
import re
c_names = ".*\.c$"

c_files = list()

for file in file_names:
    if re.match(c_names, file):
        c_files.append(file)

print(c_files)

['ftrace.c', 'tsacct.c', 'lock_events.c', 'latencytop.c', 'trace_events_hist.c', 'ntp.c', 'rcuscale.c', 'trace_sched_switch.c', 'callchain.c', 'swiotlb.c', 'prog_iter.c', 'common.c', 'arraymap.c', 'msi.c', 'trace.c', 'gcc_4_7.c', 'timekeeping_debug.c', 'iomem.c', 'kvm.c', 'braille.c', 'kexec_core.c', 'test_udelay.c', 'bpf_inode_storage.c', 'energy_model.c', 'trace_kprobe_selftest.c', 'spinlock_debug.c', 'async.c', 'power-traces.c', 'utsname.c', 'queue_stack_maps.c', 'dispatcher.c', 'rseq.c', 'timer.c', 'trace_irqsoff.c', 'trace_events.c', 'trace_nop.c', 'net_namespace.c', 'module_signing.c', 'shadow.c', 'process.c', 'cputime.c', 'trace_selftest.c', 'trace_functions.c', 'exec_domain.c', 'smpboot.c', 'patch.c', 'core_sched.c', 'bounds.c', 'rdma.c', 'sys.c', 'compat.c', 'audit_watch.c', 'stop_task.c', 'usermode_driver.c', 'resource.c', 'stop_machine.c', 'timecounter.c', 'trace_events_filter.c', 'contiguous.c', 'irqdomain.c', 'devmap.c', 'audit_tree.c', 'trace_clock.c', 'kprobe_event_gen_t

In [4]:
# load all c code in a list
full_code = list()
for file in c_files:
    code = open(file, "r", encoding='utf-8')
    full_code.append(code.read())
    code.close()

In [5]:
# let's look at how a typical C code looks like
print(full_code[20]) 







<!DOCTYPE html>
<html lang="en" data-color-mode="auto" data-light-theme="light" data-dark-theme="dark">
  <head>
    <meta charset="utf-8">
  <link rel="dns-prefetch" href="https://github.githubassets.com">
  <link rel="dns-prefetch" href="https://avatars.githubusercontent.com">
  <link rel="dns-prefetch" href="https://github-cloud.s3.amazonaws.com">
  <link rel="dns-prefetch" href="https://user-images.githubusercontent.com/">
  <link rel="preconnect" href="https://github.githubassets.com" crossorigin>
  <link rel="preconnect" href="https://avatars.githubusercontent.com">



  <link crossorigin="anonymous" media="all" integrity="sha512-dkuYFW+ra8yYSt342e5pJEeslPSjMcrMvNxlYZMyM/X+/WJHDPvoCuGq3LFojI7B0dQWwZNRiPMnbi9IfUgTaA==" rel="stylesheet" href="https://github.githubassets.com/assets/light-764b98156fab6bcc984addf8d9ee6924.css" /><link crossorigin="anonymous" media="all" integrity="sha512-UrAu23+eyncWvaQFwsLbgSKtmLb2aH1bcT4hJnnRdkaPuY1eu9bumt33FyHHFDX8hskTUNWNkIsMCz7FWQQHwA==" re

In [6]:
# merge different c codes into one big c code
text = "\n".join(full_code)
print("Total number of characters in entire code: {}".format(len(text)))

Total number of characters in entire code: 175556531


In [7]:
# top_n: only consider first top_n characters and discard the rest for memory and computational efficiency
top_n = 400000
text = text[:top_n]

## Convert characters to integers

In [8]:
# create character to index mapping
chars = sorted(list(set(text)))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

In [9]:
print("Vocabulary size: {}".format(len(chars)))

Vocabulary size: 102


## Divide data in input (X) and output (y)

### Create sequences

In [10]:
# define length for each sequence
MAX_SEQ_LENGTH = 50          # number of input characters (X) in each sequence 
STEP           = 3           # increment between each sequence
VOCAB_SIZE     = len(chars)  # total number of unique characters in dataset

sentences  = []              # X
next_chars = []              # y

for i in range(0, len(text) - MAX_SEQ_LENGTH, STEP):
    sentences.append(text[i: i + MAX_SEQ_LENGTH])
    next_chars.append(text[i + MAX_SEQ_LENGTH])

In [11]:
print('Number of training samples: {}'.format(len(sentences)))

Number of training samples: 133317


## Create input and output using the created sequences

When you're not using the Embedding layer of the Keras as the very first layer, you need to convert your data in the following format:
#### input shape should be of the form :  (#samples, #timesteps, #features)
#### output shape should be of the form :  (#samples, #timesteps, #features)

![Tensor shape](./jupyter resources/rnn_tensor.png)

#samples: the number of data points (or sequences)
#timesteps: It's the length of the sequence of your data (the MAX_SEQ_LENGTH variable).
#features: Number of features depends on the type of problem. In this problem, #features is the vocabulary size, that is, the dimensionality of the one-hot encoding matrix using which each character is being represented. If you're working with **images**, features size will be equal to: (height, width, channels), and the input shape will be (#training_samples, #timesteps, height, width, channels)

In [12]:
# create X and y
X = np.zeros((len(sentences), MAX_SEQ_LENGTH, VOCAB_SIZE), dtype=np.bool)
y = np.zeros((len(sentences), VOCAB_SIZE), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

In [13]:
print("Shape of X: {}".format(X.shape))
print("Shape of y: {}".format(y.shape))

Shape of X: (133317, 50, 102)
Shape of y: (133317, 102)


Here, X is reshaped to (#samples, #timesteps, #features). We have explicitly mentioned the third dimension (#features) because we won't use the Embedding() layer of Keras in this case since there are only 97 characters. Characters can be represented as one-hot encoded vector. There are no word embeddings for characters.

# 2. LSTM

In [26]:
# define model architecture - using a two-layer LSTM with 128 LSTM cells in each layer
model = Sequential()
model.add(LSTM(128, input_shape=(MAX_SEQ_LENGTH, VOCAB_SIZE), return_sequences=True, dropout=0.2))
model.add(LSTM(128, dropout=0.2))
model.add(Dense(VOCAB_SIZE, activation = "softmax"))

optimizer = Adam(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics = ['acc'])

In [27]:
# check model summary
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_8 (LSTM)               (None, 50, 128)           118272    
                                                                 
 lstm_9 (LSTM)               (None, 128)               131584    
                                                                 
 dense_4 (Dense)             (None, 102)               13158     
                                                                 
Total params: 263,014
Trainable params: 263,014
Non-trainable params: 0
_________________________________________________________________


In [28]:
# fit model
model.fit(X, y, batch_size=128, epochs=20)

Epoch 1/20
1042/1042 [==============================] - 162s 152ms/step - loss: 1.7422 - acc: 0.5414
Epoch 2/20
1042/1042 [==============================] - 161s 155ms/step - loss: 1.1062 - acc: 0.6997
Epoch 3/20
1042/1042 [==============================] - 160s 154ms/step - loss: 0.9913 - acc: 0.7261
Epoch 4/20
1042/1042 [==============================] - 163s 156ms/step - loss: 0.9475 - acc: 0.7373
Epoch 5/20
1042/1042 [==============================] - 158s 151ms/step - loss: 0.8997 - acc: 0.7476
Epoch 6/20
1042/1042 [==============================] - 159s 152ms/step - loss: 0.8798 - acc: 0.7521
Epoch 7/20
1042/1042 [==============================] - 158s 151ms/step - loss: 1.3243 - acc: 0.6359
Epoch 8/20
1042/1042 [==============================] - 158s 152ms/step - loss: 0.9876 - acc: 0.7235
Epoch 9/20
1042/1042 [==============================] - 164s 158ms/step - loss: 0.9065 - acc: 0.7441
Epoch 10/20
1042/1042 [==============================] - 163s 156ms/step - loss: 0.8719 - a

# 3. Generate code

Create a function that will make next character predictions based on temperature. If temperature is greater than 1, the generated characters will be more versatile and diverse. On the other hand, if temperature is less than one, the generated characters will be much more conservative.

In [29]:
# define function to sample next word from a probability array based on temperature
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [30]:
np.random.multinomial(10, [0.05, 0.9, 0.05], size=2)

array([[ 0,  9,  1],
       [ 0, 10,  0]])

In [31]:
# generate code

start_index = random.randint(0, len(text) - MAX_SEQ_LENGTH - 1) # pick random code to start text generation

for diversity in [0.5, 1.0, 1.5]:
        print('-'*50, 'diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + MAX_SEQ_LENGTH]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(1000):
            x_pred = np.zeros((1, MAX_SEQ_LENGTH, VOCAB_SIZE))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()

-------------------------------------------------- diversity: 0.5
----- Generating with seed: "ositories-ff444806.js"></script>
<script crossorig"
ositories-ff444806.js"></script>
<script crossorigin="anonymous" defer="defer" integrity="sha512-1FV7wV8hQicwNaX7Jx7wCJw0QIxM2l8800Jc28aYRMiw==" type="application/javascript" data-module-id="./chunk-form.js" data-scc="https://github.githubassets.com/assets/chunk-selector-button-libe.js" data-src="https://github.githubassets.com/assets/chunk-starss" aria-label="command-palette-item-group-header data-src="https://github.githubassets.com/assets/chunk-search-inem-starss" data-src="https://github.githubassets.com/assets/chunk-container btn-order-strrch" aria-label="Corvalds/linux masters_medese/search-contribut flex-shrenk color-bg-subtle-parder-item relain menor-color-bg-subtle-page-temp-tempors-command-palette-item-grarch resosition-search navidation-item-starserlinenav-ofel octicon-border-barder p--1 js-mump-to-suggestion-form.repository-contr

In [32]:
# generate code

start_index = random.randint(0, len(text) - MAX_SEQ_LENGTH - 1) # pick random seed

for diversity in [0.5, 1.0, 1.5]:
        print('-'*50, 'diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + MAX_SEQ_LENGTH]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(1000):
            x_pred = np.zeros((1, MAX_SEQ_LENGTH, VOCAB_SIZE))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()

-------------------------------------------------- diversity: 0.5
----- Generating with seed: ">
                  </span>
                  <div"
>
                  </span>
                  <div class="js-stroke to d-none flex-items-cherk color-fg-muted: href="/torvalds/linux/peate-scoles/logtersorifars" data-sepp-types="{&quot;ewper&quot;:&quot;icon_ctask&quot;}, {&quot;experimentIds&quot;: [], &quot;id&quot;: &quot;202947019&quot;, &quot;key&quot;: &quot;click.artary_url_urd_cran_reg_rerosity_quot;}, {&quot;experimentIds&quot;: [], &quot;id&quot;: &quot;2059970515&quot;, &quot;key&quot;: &quot;205414543&quot;, &quot;key&quot;: &quot;2049454663&quot;], &quot;id&quot;: &quot;204317316&quot;, &quot;key&quot;: &quot;2046647472&quot;, &quot;key&quot;: &quot;click.ser_cll_nerdering_url_urce_tege_stacks&quot;}, {&quot;experimentIds&quot;: [], &quot;id&quot;: &quot;209873357&quot;, &quot;key&quot;: &quot;click.chers_ies_oction_cles_list_oname_urg_repository_searoc sesitions page in starte